### basic model eval and compare - use static pretrained global embeddings + length + get cv preds
* note:

* all seqs: filtered for under length 15xx
* viruses = Swissprot reviewed. + host is an animal (human, vertebrata, mammal)
* human: SWP.

* redundnants filtered. May be ur90 (was filtered also to dorp identicals in first 300 positions + 188 alphabets). Human seqs maps to same # of seqs in UR90 as here ,so may be ur90 already.
* Train test splits done by UR50

In [1]:
import pandas as pd
import numpy as np
import random

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing  import MinMaxScaler,StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score, log_loss, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier   
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedGroupKFold,GroupKFold
from sklearn.compose import make_column_selector as selector
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.compose import make_column_selector as selector

# from tqdm.autonotebook import tqdm
import shap

In [2]:
NUMERIC_ONLY = False # use only numbers, inore raw sequence
FAST_RUN = False#True
# FAST_RUN = True

basic_keep_cols_list = ['Length',  'Sequence'] # "av_mw" # we exclude av_mw fom the data here, manually on import
# 'Mass', 
SAVE_CV_MISTAKE_PREDS = False#True
SAVE_EVAL_RES = False#True

In [3]:
def eval_model(clf,X_train,y_train,groups,numCV=3,X_test=None,y_test=None,return_preds=True,return_test_preds=True):
    train_cv_preds = cross_val_predict(estimator=clf, X=X_train, y=y_train,
                                       # cv=GroupKFold(n_splits=numCV), # ORIG
                                       cv=StratifiedGroupKFold(n_splits=numCV),
                                       groups=groups,method="predict_proba",n_jobs=-2)
    print("CV-Train res")
    print(classification_report(y_train,train_cv_preds[:,1]>=.5))
    print("AUC",roc_auc_score(y_train,train_cv_preds[:,1]).round(5))
    
    if X_test is not None:
        clf.fit(X_train,y_train)
        y_test_pred = clf.predict_proba(X_test)
        print("TEST res")
        print(classification_report(y_test,y_test_pred[:,1]>=.5))
        print("Test AUC",roc_auc_score(y_test,y_test_pred[:,1]).round(5))
    
    if return_preds:
        if return_test_preds:
            return y_test_pred[:,1]
        else:
            return train_cv_preds[:,1]


def get_model_results(model_name:str,y_test,y_pred):
    """
    Note: returns macro averaged prec/recall/f1.
    >>> df_res = pd.DataFrame()
    >>> df_res = pd.concat([df_res, pd.DataFrame.from_dict(get_model_results("Length model",y_test=y_test,y_pred=length_test_preds),orient="index").T], ignore_index=True)
    """
    # classification_report, roc_auc_score, log_loss, precision_recall_fscore_support, accuracy_score
    AUC = roc_auc_score(y_test,y_pred)
    # res = classification_report(y_test,y_pred>=0.5,output_dict=True)#.keys()
    precision, recall, F1, _ = precision_recall_fscore_support(y_test,y_pred>=0.5,average="macro") # micro used in DL eval
    acc = accuracy_score(y_test,y_pred>=0.5)

    return {"model":model_name,
            "eval_roc_auc":AUC,
    "eval_accuracy":acc,
    "eval_precision":precision, "eval_recall":recall,
            "eval_loss":log_loss(y_test,y_pred),
            "eval_f1":F1}

In [4]:
X_train = pd.read_csv("hum_vir_swp-globalEmbed-train.csv.gz",nrows= 700 if FAST_RUN else 1e9)
# display(X_train.iloc[:,-10:].groupby(["virus"]).select_dtypes(["number"]).agg(["mean","max","min"]).round(2))
y_train = X_train.pop("virus")
train_groups = X_train["Cluster name"]
X_train.drop(columns=['Mass', "av_mw"],errors="ignore",inplace=True)
keep_feature_cols_list = list(X_train.select_dtypes(["number","bool"]).columns)
if not NUMERIC_ONLY:
    keep_feature_cols_list = keep_feature_cols_list + ["Sequence"]
    
X_train = X_train.filter(keep_feature_cols_list,axis=1)
print(X_train.shape)
print(y_train.mean().round(3))
display(X_train)

(20340, 1026)
0.279


,global_embed_0,global_embed_1,global_embed_2,global_embed_3,global_embed_4,global_embed_5,global_embed_6,global_embed_7,global_embed_8,global_embed_9,...,global_embed_1016,global_embed_1017,global_embed_1018,global_embed_1019,global_embed_1020,global_embed_1021,global_embed_1022,global_embed_1023,Length,Sequence
0,0.040560,0.012460,0.04860,0.006645,-0.006474,0.027070,0.019240,-0.10620,-0.005110,0.01721,...,0.018650,-0.004280,0.021000,0.002610,-0.004677,0.019590,-0.02592,-0.026840,102,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...
1,0.007630,-0.004948,0.02852,-0.001821,-0.039600,0.066800,-0.020630,-0.16850,0.003940,-0.02774,...,-0.059500,-0.019410,0.041660,0.047820,-0.034120,0.008250,-0.00011,0.052370,362,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...
2,0.037300,-0.061070,0.08730,0.005596,-0.008150,0.000575,0.011734,-0.16220,-0.013660,-0.02573,...,-0.014534,-0.033780,0.033630,-0.093140,0.093300,0.026840,0.08120,0.024750,97,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...
3,-0.002218,0.002700,0.07270,-0.022480,0.001690,0.040280,0.001864,-0.17580,0.098200,0.03928,...,0.064760,-0.070200,-0.032100,0.000459,0.019100,0.055500,0.11730,-0.012960,186,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...
4,0.069760,0.098600,0.04890,0.004288,0.022430,0.062700,-0.037500,-0.05383,-0.027130,-0.02348,...,-0.058800,0.008675,0.039180,-0.004436,-0.029570,0.034200,-0.01112,-0.009920,1360,MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20335,-0.041050,-0.032000,0.03632,-0.013990,-0.042300,0.070900,-0.043600,-0.06573,0.044250,0.03620,...,0.008830,-0.029920,-0.006527,0.036900,0.000668,-0.010840,-0.01273,0.018340,578,MDPDKQDALNSIENSIYRTAFKLQSVQTLCQLDLIDSSLIQQVLLR...
20336,0.016450,-0.008040,0.04852,0.029430,-0.040300,0.040100,-0.041870,-0.11350,0.047270,0.03058,...,0.033300,-0.091400,0.001120,-0.027710,-0.014870,-0.000851,0.06420,0.044980,348,MLCPWRTANLGLLLILTIFLVAEAEGAAQPNNSLMLQTSKENHALA...
20337,-0.072300,0.009674,0.08984,0.002928,-0.052500,0.059480,-0.031600,-0.07390,0.024300,0.03186,...,-0.019200,-0.080140,0.008230,0.033750,-0.027820,0.004436,0.06400,0.001059,421,MCLRFFSPVPGSTSSATNVTMVVSAGPWSSEKAEMNILEINEKLRP...
20338,0.009220,-0.034030,0.01591,-0.005898,0.019420,0.008280,-0.054140,-0.07000,0.005203,-0.04065,...,0.023390,-0.068000,-0.036100,-0.056500,-0.011086,-0.031430,0.05466,0.043950,376,MASHAGQQHAPAFGQAARASGPTDGRAASRPSHRQGASEARGDPEL...


In [5]:
print("Test")
X_test = pd.read_csv("hum_vir_swp-globalEmbed-test.csv.gz",nrows= 500 if FAST_RUN else 1e9)
X_test.drop(columns=['Mass', "av_mw"],errors="ignore",inplace=True)
test_groups = X_test["Cluster name"] # these aren't used , but nice to have to sanity check
X_test.drop(columns=["Cluster name"],errors="ignore",inplace=True)
y_test = X_test.pop("virus")

X_test = X_test.filter(keep_feature_cols_list,axis=1)

print(X_test.shape)
print(y_test.mean().round(3))

Test
(4777, 1026)
0.215


In [6]:
print(X_train.shape[0]+X_test.shape[0])

25117


In [7]:
print(test_groups.nunique())
assert len ([i for i in train_groups if i in test_groups])==0 # no overlap between groups

4488


In [8]:
test_groups.value_counts()

Cluster name
Cluster: Nucleoprotein                                            20
Cluster: Phosphoprotein                                           10
Cluster: Envelope glycoprotein N                                   9
Cluster: Matrix protein                                            9
Cluster: Thymidine kinase                                          8
                                                                  ..
Cluster: Exosome component 10                                      1
Cluster: Putative uncharacterized protein encoded by LINC00476     1
Cluster: Protocadherin-9                                           1
Cluster: Myc proto-oncogene protein                                1
Cluster: Centrosomal protein of 76 kDa                             1
Name: count, Length: 4488, dtype: int64

In [9]:
X_test.isna().max(axis=0).sum() # no missing values

0

In [10]:
X_test

,global_embed_0,global_embed_1,global_embed_2,global_embed_3,global_embed_4,global_embed_5,global_embed_6,global_embed_7,global_embed_8,global_embed_9,...,global_embed_1016,global_embed_1017,global_embed_1018,global_embed_1019,global_embed_1020,global_embed_1021,global_embed_1022,global_embed_1023,Length,Sequence
0,0.025620,-0.002438,0.045720,0.04990,-0.003360,0.001394,-0.055600,-0.08990,0.032170,-0.015594,...,0.01883,-0.045720,-0.018170,-0.019180,-0.007690,0.015720,0.005882,-0.01941,102,MTTATPLGDTTFFSLNMTTRGEDFLYKSSGAIVAAVVVVVIIIFTV...
1,0.048920,0.053920,0.037050,0.02147,-0.003021,0.049130,-0.021260,-0.08130,0.018370,-0.018070,...,-0.01804,0.007990,0.061000,-0.004280,-0.029430,0.002352,-0.036800,0.03096,907,MESAIAEGGASRFSASSGGGGSRGAPQHYPKTAGNSEFLGKTPGQN...
2,0.038100,0.071840,0.037840,0.02646,-0.009040,0.037170,-0.031280,-0.09094,0.030550,-0.015300,...,0.02495,-0.013080,-0.014190,-0.046360,0.005936,-0.009346,-0.012470,-0.03450,329,MGTAAAAAAAAAAAAAGEGARSPSPAAVSLGLGVAVVSSLVNGSTF...
3,-0.015030,-0.016920,0.010445,0.01985,-0.016340,0.039060,-0.031070,-0.05377,0.024690,0.009270,...,0.00788,-0.039860,0.047820,0.048770,0.029710,-0.018880,0.059140,0.00568,460,MFTIKLLLFIVPLVISSRIDQDNSSFDSLSPEPKSRFAMLDDVKIL...
4,-0.000349,0.021680,0.151700,0.02565,-0.020340,0.090100,0.021700,-0.14670,-0.029890,0.134600,...,-0.01367,-0.044130,-0.034180,-0.040900,-0.085140,0.024300,0.023480,0.03925,347,MPRGQKSKLRAREKRRKAREETQGLKVAHATAAEKEECPSSSPVLG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,-0.050000,-0.029130,0.075600,0.01599,-0.022140,0.042660,0.026350,-0.07434,-0.000616,0.006010,...,0.03177,-0.040620,-0.041930,0.026470,0.006435,0.059500,-0.001407,0.10913,164,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...
4773,0.008865,-0.047000,0.015396,0.04404,-0.038450,0.085270,-0.061100,-0.06018,0.085200,0.066100,...,0.01819,-0.001308,-0.001677,-0.011765,-0.010510,0.019680,0.049000,0.05966,1167,MKVNRETKRLYVGGLSQDISEADLQNQFSRFGEVSDVEIITRKDDQ...
4774,0.042570,0.050170,0.009445,0.02354,0.011020,0.076050,-0.026640,-0.06350,0.017720,0.000740,...,0.02773,0.016720,0.036040,-0.008484,-0.009030,0.015730,0.020370,0.04540,567,MAGVGDAAAPGEGGGGGVDGPQRDGRGEAEQPGGSGGQGPPPAPQL...
4775,-0.017070,0.005505,0.044560,-0.01119,-0.005690,0.047180,0.032840,-0.07556,0.018000,-0.002062,...,0.04550,0.009690,-0.003290,-0.053900,0.005430,-0.048580,-0.021910,0.03400,283,MGASVSRGRAARVPAPEPEPEEALDLSQLPPELLLVVLSHVPPRTL...


### store test set results for models.

In [11]:
# test_res_dict = {} # store results on test set, per model
# "eval_loss","eval_accuracy","eval_f1","eval_precision","eval_recall","eval_roc_auc"
## store test set results for models. Will add DL models seperately/manually? 
df_res = pd.DataFrame()

In [12]:
##### was in "not NUMERIC_ONLY" orig
# Identify numerical and text features

## could replace num with make_column_selector(dtype_include=np.number)  # https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
num_features = [i for i in X_train.columns if i != 'Sequence']
text_features = ['Sequence']

# For numerical features, we use a simple identity function
#     num_transformer = FunctionTransformer(func=lambda x: x, validate=True)
num_transformer = StandardScaler()

# For the text feature, we use TfidfVectorizer with char-level n-grams
text_transformer = Pipeline(steps=[
('tfidf',TfidfVectorizer(analyzer='char', ngram_range=(1, 3),
                                   min_df=5, max_df=0.97,
                                   lowercase=False,
                         # max_features=50 if FAST_RUN else 5_000
                        )),
#     ('chi2', SelectKBest(chi2, k=3000))  # Select 20 best features
])
#####

# LR = LogisticRegressionCV(n_jobs=-1,)#max_iter=200,class_weight="balanced"))

# Build ColumnTransformer for preprocessing step of clf
preprocessor = ColumnTransformer(transformers=[
#     ('num', num_transformer, num_features),# ORIG
    ("num", num_transformer, selector(dtype_include="number")), # new
    ('text', text_transformer, text_features[0])
],remainder="passthrough",sparse_threshold=0)


if NUMERIC_ONLY:
    clf = HistGradientBoostingClassifier()
    clf_lin = Pipeline(steps=[
        ('preprocessor', StandardScaler()),
         # ('classifier',LogisticRegression()#CV(n_jobs=-1)#,max_iter=200,class_weight="balanced")
          ('classifier',LogisticRegressionCV(n_jobs=-2)
         )])
else:  
    # Create clf
    clf_lin = Pipeline(steps=[
        ('preprocessor', preprocessor),
         ('classifier',
          # LogisticRegression()#CV(n_jobs=-2)
          LogisticRegressionCV(n_jobs=-2)
         ) 
    ])

    clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
#         ('classifier', RandomForestClassifier(n_jobs=-1,)#GradientBoostingClassifier() ## gbc is slow 
     # min_samples_leaf=2,
     # hist gbc - doesn't suppot sparse? 
     ('classifier',HistGradientBoostingClassifier()
     )

])

## basic = for numeric only/length only
clf_lin_basic = Pipeline(steps=[
        ('preprocessor', StandardScaler()),
         ('classifier',LogisticRegressionCV()#CV(n_jobs=-1)#,max_iter=200,class_weight="balanced")
         )])

## AA text features + any numeric features left in, without scaling them 
aa_preprocessor = ColumnTransformer(transformers=[
#         ('num', num_transformer, num_features),
    ("num", num_transformer, selector(dtype_include="number")),
        ('text', text_transformer, text_features[0])
    ],remainder="passthrough",sparse_threshold=0)

clf_aa = Pipeline(steps=[
    ('preprocessor', aa_preprocessor),
     ('classifier',HistGradientBoostingClassifier()
     )

])


# # Fit the model
# clf.fit(X_train, y_train)
# print("Trained")

In [13]:
# %%time
# ##slow? 

# ## # Get the feature names extracted by TF-IDF
# ## feature_names = num_features + preprocessor.transformers_[1][1].get_feature_names_out()
# #ALT: (with FS)# Get the feature names extracted by TF-IDF and chi2
# feature_names = num_features + [f'text_{i}' for i in range(text_transformer.named_steps['chi2'].k)]

# # Create SHAP explainer
# explainer = shap.TreeExplainer(clf.named_steps['classifier'])
# print("Explainer")
# # Calculate SHAP values
# shap_values = explainer.shap_values(preprocessor.transform(X_test.head(55)))

# # Summarize the SHAP values in a pandas DataFrame for better visualization
# shap_df = pd.DataFrame(shap_values[0], columns=feature_names)

# # Print SHAP values
# # print(shap_df)

# shap_values = explainer(preprocessor.transform(X_test.head(55)),y_test.head(55))

# # # visualize the first prediction's explanation
# # shap.plots.waterfall(shap_values[0])

# # shap.plots.beeswarm(shap_values)
# ## hack for random forest:
# shap.plots.beeswarm(shap_values[:,:,1]) ## laks column/feature names..

#### length, AA baseline models

In [14]:
length_test_preds = eval_model(clf=clf_lin_basic,X_train=X_train["Length"].values.reshape(-1, 1),y_train=y_train,groups=train_groups,
           numCV=3,X_test=X_test["Length"].values.reshape(-1, 1),y_test=y_test)
# length_test_preds = eval_model(clf=clf_lin_basic,X_train=X_train["Length"],y_train=y_train,groups=train_groups,
#            numCV=3,X_test=X_test["Length"],y_test=y_test)


df_res = pd.concat([df_res, pd.DataFrame.from_dict(get_model_results("Length model",y_test=y_test,
                                                                     y_pred=length_test_preds),orient="index").T], ignore_index=True)

CV-Train res
              precision    recall  f1-score   support

           0       0.72      1.00      0.84     14668
           1       0.00      0.00      0.00      5672

    accuracy                           0.72     20340
   macro avg       0.36      0.50      0.42     20340
weighted avg       0.52      0.72      0.60     20340

AUC 0.60538


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


TEST res
              precision    recall  f1-score   support

           0       0.79      1.00      0.88      3750
           1       0.00      0.00      0.00      1027

    accuracy                           0.79      4777
   macro avg       0.39      0.50      0.44      4777
weighted avg       0.62      0.79      0.69      4777

Test AUC 0.61975


Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


In [15]:
# clf_aa.fit(X_train[basic_keep_cols_list],y_train) # works in fast run

In [16]:
%%time
## AA n-grams only model

aa_test_cv_preds = eval_model(clf=clf_aa,X_train=X_train[basic_keep_cols_list],y_train=y_train,groups=train_groups,
           numCV=2,X_test=X_test[basic_keep_cols_list],y_test=y_test)

df_res = pd.concat([df_res,
                    pd.DataFrame.from_dict(get_model_results("AA model",y_test=y_test,
                                                             y_pred=aa_test_cv_preds),orient="index").T], ignore_index=True)

CV-Train res
              precision    recall  f1-score   support

           0       0.83      0.96      0.89     14668
           1       0.84      0.49      0.62      5672

    accuracy                           0.83     20340
   macro avg       0.84      0.73      0.76     20340
weighted avg       0.83      0.83      0.82     20340

AUC 0.88871
TEST res
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      3750
           1       0.80      0.60      0.69      1027

    accuracy                           0.88      4777
   macro avg       0.85      0.78      0.81      4777
weighted avg       0.88      0.88      0.88      4777

Test AUC 0.91793
CPU times: user 5min 16s, sys: 29.5 s, total: 5min 46s
Wall time: 2min 50s


In [17]:
%%time
test_cv_preds = eval_model(clf=clf,X_train=X_train,y_train=y_train,groups=train_groups,
           numCV=2,X_test=X_test,y_test=y_test)

df_res = pd.concat([df_res,
                    pd.DataFrame.from_dict(get_model_results("Tree-AllFeat",
                                                             y_test=y_test,y_pred=test_cv_preds),orient="index").T], ignore_index=True)

CV-Train res
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     14668
           1       0.95      0.90      0.92      5672

    accuracy                           0.96     20340
   macro avg       0.95      0.94      0.95     20340
weighted avg       0.96      0.96      0.96     20340

AUC 0.99102
TEST res
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3750
           1       0.93      0.96      0.94      1027

    accuracy                           0.98      4777
   macro avg       0.96      0.97      0.96      4777
weighted avg       0.98      0.98      0.98      4777

Test AUC 0.99559
CPU times: user 5min 44s, sys: 33.6 s, total: 6min 18s
Wall time: 3min


In [18]:
%%time
print("Tree-DeepFeat")
test_cv_preds =eval_model(clf=HistGradientBoostingClassifier(),
                          X_train=X_train.drop(columns=basic_keep_cols_list),y_train=y_train,groups=train_groups,
           numCV=2,X_test=X_test.drop(columns=basic_keep_cols_list),y_test=y_test,return_preds=True)

df_res = pd.concat([df_res,
                    pd.DataFrame.from_dict(get_model_results("Tree-DeepFeat",
                                                             y_test=y_test,y_pred=test_cv_preds),orient="index").T], ignore_index=True)

Tree-DeepFeat
CV-Train res
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     14668
           1       0.95      0.89      0.92      5672

    accuracy                           0.96     20340
   macro avg       0.95      0.94      0.94     20340
weighted avg       0.96      0.96      0.96     20340

AUC 0.99093
TEST res
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3750
           1       0.93      0.95      0.94      1027

    accuracy                           0.98      4777
   macro avg       0.96      0.97      0.96      4777
weighted avg       0.98      0.98      0.98      4777

Test AUC 0.99601
CPU times: user 25.9 s, sys: 431 ms, total: 26.4 s
Wall time: 13.6 s


In [19]:
%%time
test_cv_preds_linear =eval_model(clf=clf_lin,X_train=X_train,y_train=y_train,groups=train_groups,
           numCV=2,X_test=X_test,y_test=y_test,return_preds=True)

df_res = pd.concat([df_res,
                    pd.DataFrame.from_dict(get_model_results("Linear-AllFeat",
                                                             y_test=y_test,y_pred=test_cv_preds_linear),orient="index").T], ignore_index=True)

CV-Train res
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     14668
           1       0.96      0.93      0.94      5672

    accuracy                           0.97     20340
   macro avg       0.96      0.96      0.96     20340
weighted avg       0.97      0.97      0.97     20340

AUC 0.99361
TEST res
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3750
           1       0.94      0.95      0.94      1027

    accuracy                           0.98      4777
   macro avg       0.96      0.97      0.96      4777
weighted avg       0.98      0.98      0.98      4777

Test AUC 0.99563
CPU times: user 1min 8s, sys: 10.9 s, total: 1min 19s
Wall time: 3min 37s


In [20]:
%%time
print("Lin model, without length and similars")
test_cv_preds_linear =eval_model(clf=clf_lin_basic,X_train=X_train.drop(columns=basic_keep_cols_list),y_train=y_train,groups=train_groups,
           numCV=2,X_test=X_test.drop(columns=basic_keep_cols_list),y_test=y_test,return_preds=True)

df_res = pd.concat([df_res,
                    pd.DataFrame.from_dict(get_model_results("Linear-DeepFeat",
                                                             y_test=y_test,y_pred=test_cv_preds_linear),orient="index").T], ignore_index=True)

Lin model, without length and similars
CV-Train res
              precision    recall  f1-score   support

           0       0.97      0.98      0.98     14668
           1       0.96      0.93      0.94      5672

    accuracy                           0.97     20340
   macro avg       0.96      0.96      0.96     20340
weighted avg       0.97      0.97      0.97     20340

AUC 0.9936
TEST res
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3750
           1       0.94      0.95      0.94      1027

    accuracy                           0.98      4777
   macro avg       0.96      0.97      0.96      4777
weighted avg       0.98      0.98      0.98      4777

Test AUC 0.99561
CPU times: user 2min 53s, sys: 56.7 s, total: 3min 49s
Wall time: 23.2 s


In [21]:
df_res = 100*df_res.sort_values("eval_roc_auc").convert_dtypes().set_index("model")
df_res["eval_loss"] = df_res["eval_loss"]/100
df_res =df_res.convert_dtypes().round(2).reset_index()
df_res.round(2)

,model,eval_roc_auc,eval_accuracy,eval_precision,eval_recall,eval_loss,eval_f1
0,Length model,61.97,78.5,39.25,50.0,0.53,43.98
1,AA model,91.79,88.15,84.7,78.07,0.29,80.68
2,Tree-AllFeat,99.56,97.59,95.96,97.02,0.07,96.47
3,Linear-DeepFeat,99.56,97.57,96.28,96.54,0.06,96.41
4,Linear-AllFeat,99.56,97.57,96.28,96.54,0.06,96.41
5,Tree-DeepFeat,99.6,97.57,96.08,96.79,0.06,96.43


In [22]:
df_res = 100*df_res.sort_values("eval_roc_auc").convert_dtypes().set_index("model")
df_res["eval_loss"] = df_res["eval_loss"]/100
df_res =df_res.convert_dtypes().round(2).reset_index()
df_res.round(2)

,model,eval_roc_auc,eval_accuracy,eval_precision,eval_recall,eval_loss,eval_f1
0,Length model,6197.0,7850.0,3925.0,5000.0,0.53,4398.0
1,AA model,9179.0,8815.0,8470.0,7807.0,0.29,8068.0
2,Tree-AllFeat,9956.0,9759.0,9596.0,9702.0,0.07,9647.0
3,Linear-DeepFeat,9956.0,9757.0,9628.0,9654.0,0.06,9641.0
4,Linear-AllFeat,9956.0,9757.0,9628.0,9654.0,0.06,9641.0
5,Tree-DeepFeat,9960.0,9757.0,9608.0,9679.0,0.06,9643.0


In [23]:
df_res.convert_dtypes().set_index("model").round(2)

,eval_roc_auc,eval_accuracy,eval_precision,eval_recall,eval_loss,eval_f1
model,,,,,,
Length model,6197.0,7850.0,3925.0,5000.0,0.53,4398.0
AA model,9179.0,8815.0,8470.0,7807.0,0.29,8068.0
Tree-AllFeat,9956.0,9759.0,9596.0,9702.0,0.07,9647.0
Linear-DeepFeat,9956.0,9757.0,9628.0,9654.0,0.06,9641.0
Linear-AllFeat,9956.0,9757.0,9628.0,9654.0,0.06,9641.0
Tree-DeepFeat,9960.0,9757.0,9608.0,9679.0,0.06,9643.0


In [24]:
df_res

,model,eval_roc_auc,eval_accuracy,eval_precision,eval_recall,eval_loss,eval_f1
0,Length model,6197.0,7850.0,3925.0,5000.0,0.53,4398.0
1,AA model,9179.0,8815.0,8470.0,7807.0,0.29,8068.0
2,Tree-AllFeat,9956.0,9759.0,9596.0,9702.0,0.07,9647.0
3,Linear-DeepFeat,9956.0,9757.0,9628.0,9654.0,0.06,9641.0
4,Linear-AllFeat,9956.0,9757.0,9628.0,9654.0,0.06,9641.0
5,Tree-DeepFeat,9960.0,9757.0,9608.0,9679.0,0.06,9643.0


In [25]:
if SAVE_EVAL_RES:
    df_res.to_csv("./outputs/Table-2-modelsEval.csv",index=False)

* ensembl preds - doesn't improve

In [22]:
## similar or slightly better perf - negligible difference
joint_preds = (test_cv_preds_linear + test_cv_preds)/2
print(joint_preds.shape)
print(classification_report(y_test,joint_preds>=.5))
print("AUC",roc_auc_score(y_test,joint_preds).round(4))

# ## overwrite old preds
# train_cv_preds = joint_preds

(4777,)
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      3750
           1       0.94      0.96      0.95      1027

    accuracy                           0.98      4777
   macro avg       0.96      0.97      0.97      4777
weighted avg       0.98      0.98      0.98      4777

AUC 0.9964


## CV on all - get output preds + erros
### predictions error analysis

* Q: Should we use best model (Linea + DL feats + Length ; or model without the added length feat (i.e like ap roper DL only model)?

In [23]:
X = pd.concat([X_train,X_test])
print(X.shape)
y = pd.concat([y_train,y_test])
print(y.shape)
groups = pd.concat([train_groups,test_groups])
X

(25117, 1026)
(25117,)


,global_embed_0,global_embed_1,global_embed_2,global_embed_3,global_embed_4,global_embed_5,global_embed_6,global_embed_7,global_embed_8,global_embed_9,...,global_embed_1016,global_embed_1017,global_embed_1018,global_embed_1019,global_embed_1020,global_embed_1021,global_embed_1022,global_embed_1023,Length,Sequence
0,-0.002075,-0.013176,0.02216,0.013760,-0.047940,0.03510,-0.005756,-0.05280,0.04092,0.036530,...,0.014010,-0.044340,0.004120,-0.003265,0.023090,0.027570,0.04825,0.021650,832,MAMRSGRHPSLLLLLVLLLWLLQVSIIDSVQQETDDLTKQTKEKIY...
1,-0.033000,0.128700,0.02954,0.025380,0.049900,-0.10693,-0.012290,-0.06854,-0.01787,-0.010735,...,-0.021700,-0.104900,0.129800,0.046260,0.030100,-0.003197,0.14530,0.070740,819,MMNLEKNFDKTTLFNHMRTDKRGKCSDLNEYGTSCDKTTAVEYNKV...
2,-0.002344,0.018860,-0.03217,-0.009346,0.048770,0.02544,-0.026600,-0.02580,0.02396,-0.044280,...,0.073700,-0.072400,0.076000,0.076000,-0.024900,0.013810,-0.08330,-0.014710,486,MATFKDACYHYKRINKLNHTVLKLGVNDTWRSSPPTKYKGWCLDCC...
3,0.014910,0.012170,0.05118,0.033970,-0.046660,0.06740,0.004925,-0.06200,0.04470,-0.041320,...,0.022250,-0.098270,0.004417,0.058600,0.001499,0.032070,-0.01502,0.002170,233,MSAKRAELKKTHLSKNYKAVCLELKPEPTKTFDYKAVKQEGRFTKA...
4,0.010605,0.080100,0.07580,0.011850,-0.018720,0.02084,-0.020360,-0.05804,0.06890,0.039030,...,0.012240,0.009476,0.019840,-0.055540,-0.000411,0.003155,0.01721,-0.025160,1049,MGSRTPESPLHAVQLRWGPRRRPPLLPLLLLLLPPPPRVGGFNLDA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,0.024860,0.077940,0.02074,0.019450,-0.041080,0.03412,-0.033700,-0.12630,0.02231,0.026170,...,-0.005222,-0.043330,0.040500,0.023210,0.005642,0.005108,0.01581,0.005238,541,MKSYTPYFILLWSAVGIAKAAKIIIVPPIMFESHMYIFKTLASALH...
4773,0.045750,0.010086,-0.00791,0.024440,-0.043180,0.07916,0.003970,-0.03190,0.04657,0.005486,...,0.053960,0.008720,0.019260,-0.018370,-0.044070,-0.006134,0.02430,0.023700,238,MAARTGHTALRRVVSGCRPKSATAAGAQAPVRNGRYLASCGILMSR...
4774,0.003473,0.036130,0.06660,0.022400,0.001187,0.00559,-0.039180,-0.05566,-0.02106,-0.003777,...,0.026120,-0.018050,0.030760,-0.022720,-0.030470,-0.053680,-0.00893,0.011220,384,MAAVQMDPELAKRLFFEGATVVILNMPKGTEFGIDYNSWEVGPKFR...
4775,0.038540,0.054930,0.01980,0.018390,-0.040560,0.02724,0.000665,-0.08060,0.00828,-0.015720,...,0.012860,-0.069030,0.018260,0.030650,0.009010,0.002167,0.02405,0.000231,235,MRDLLQYIACFFAFFSAGFLIVATWTDCWMVNADDSLEVSTKCRGL...


In [24]:
%%time
all_preds = eval_model(clf=clf_lin_basic,X_train=X.drop(columns=["Sequence"]) #.set_index("Sequence")
                       ,y_train=y,groups=groups,
           numCV=4,return_preds=True,return_test_preds=False)
all_preds

CV-Train res
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     18418
           1       0.96      0.94      0.95      6699

    accuracy                           0.97     25117
   macro avg       0.97      0.96      0.97     25117
weighted avg       0.97      0.97      0.97     25117

AUC 0.99424
CPU times: user 6.21 s, sys: 325 ms, total: 6.54 s
Wall time: 1min 33s


array([5.59194262e-04, 8.64401986e-07, 9.99998690e-01, ...,
       2.81372484e-03, 5.38118073e-03, 2.52926900e-02])

In [25]:
roc_auc_score(y,all_preds)

0.9942415023913257

In [26]:
df_preds = X.filter(X.select_dtypes(["O"]).columns.to_list() + ["Length","virus"],axis=1).copy()
df_preds["virus"] = y
df_preds["model_pred_proba_vir"] = all_preds.round(4)
df_preds["model_pred_vir"] = (all_preds>=.5).astype(int)
print("mistakes/mismatches")
df_preds["mistake"] = (df_preds["model_pred_vir"]!=df_preds["virus"])

# ## keep only mismatch cases
# df_preds = df_preds.loc[df_preds["model_pred"]!=df_preds["virus"]].sort_values(["model_pred_proba"])
print(df_preds["mistake"].agg(["sum","mean"]))
# df_preds

mistakes/mismatches
sum     682.000000
mean      0.027153
Name: mistake, dtype: float64


In [27]:
df_meta = pd.read_csv("SWP_humVir_meta_v2.csv.gz")
print(df_meta.shape)
print("nunique:")
print(df_meta.nunique())
print(df_meta.Sequence.str.len().describe().round(1))
df_meta.head(2)

(25119, 16)
nunique:
Sequence              25119
Baltimore                 6
Family                   50
Genome Composition        9
Genus                   171
Keywords              15980
Mass                  22130
Organism               1136
Protein names         21394
Taxonomic lineage       561
UR50_Cluster ID       20886
Virus hosts             291
av_mw                  2110
genus_human_host          2
group                 23473
human_host                2
dtype: int64
count    25119.0
mean       446.7
std        307.8
min         11.0
25%        213.0
50%        371.0
75%        592.0
max       1534.0
Name: Sequence, dtype: float64


,Sequence,Baltimore,Family,Genome Composition,Genus,Keywords,Mass,Organism,Protein names,Taxonomic lineage,UR50_Cluster ID,Virus hosts,av_mw,genus_human_host,group,human_host
0,AARADGTMGFSSELCSPQGHGAEQQMQEAELRLLEGMRKWMAQRVK...,ssRNA-RT,Retroviridae,ssRNA-RT,Gammaretrovirus,ATP-binding . Kinase . Nucleotide-binding . On...,68769,Feline sarcoma virus (strain Gardner-Arnstein)...,Tyrosine-protein kinase transforming protein F...,"Viruses, Riboviria, Pararnavirae (kingdom), Ar...",P00542,Felidae (cat family) [TaxID: 9681],112.92,False,P00542,False
1,ACSTPHYDLPLICAATWTATCVPRQTSAARCGPCPRHDPHPSAPLP...,dsDNA,Herpesviridae,dsDNA,Cytomegalovirus,NaN,12796,Human cytomegalovirus (strain AD169) (HHV-5) (...,Uncharacterized protein UL81,"Viruses, Duplodnaviria, Heunggongvirae (kingdo...",P16825,Homo sapiens (Human) [TaxID: 9606],110.31,True,P16825,True


In [28]:
df_preds = df_preds.merge(df_meta, on="Sequence",
                 how= "inner",#'outer',
                          suffixes=('', '_y'))

df_preds.drop(df_preds.filter(regex='_y$').columns, axis=1, inplace=True)
df_preds.drop(columns=["group"],errors="ignore",inplace=True)
df_preds

,Sequence,Length,virus,model_pred_proba_vir,model_pred_vir,mistake,Baltimore,Family,Genome Composition,Genus,Keywords,Mass,Organism,Protein names,Taxonomic lineage,UR50_Cluster ID,Virus hosts,av_mw,genus_human_host,human_host
0,MAMRSGRHPSLLLLLVLLLWLLQVSIIDSVQQETDDLTKQTKEKIY...,832,0,0.0006,0,False,NaN,NaN,NaN,NaN,Alternative splicing . Calcium . Cell adhesion...,92416,Homo sapiens (Human),Cadherin-like protein 26 (Cadherin-like protei...,"cellular organisms, Eukaryota, Opisthokonta, M...",Q8IXH8,NaN,111.08,NaN,NaN
1,MMNLEKNFDKTTLFNHMRTDKRGKCSDLNEYGTSCDKTTAVEYNKV...,819,0,0.0000,0,False,NaN,NaN,NaN,NaN,DNA-binding . Metal-binding . Nucleus . Refere...,94331,Homo sapiens (Human),Zinc finger protein 658B,"cellular organisms, Eukaryota, Opisthokonta, M...",Q4V348,NaN,115.18,NaN,NaN
2,MATFKDACYHYKRINKLNHTVLKLGVNDTWRSSPPTKYKGWCLDCC...,486,1,1.0000,1,False,dsRNA,Sedoreoviridae,dsRNA,Rotavirus,Host cytoplasm . Host cytoskeleton . Host-viru...,57672,Rotavirus A (strain RVA/Human/United States/P/...,Non-structural protein 1 (NSP1) (NCVP2) (Non-s...,"Viruses, Riboviria, Orthornavirae (kingdom), D...",NaN,Homo sapiens (Human) [TaxID: 9606],118.67,True,True
3,MSAKRAELKKTHLSKNYKAVCLELKPEPTKTFDYKAVKQEGRFTKA...,233,0,0.2340,0,False,NaN,NaN,NaN,NaN,Alternative splicing . Coiled coil . Reference...,26518,Homo sapiens (Human),Testis-expressed protein 35,"cellular organisms, Eukaryota, Opisthokonta, M...",Q5T0J7,NaN,113.81,NaN,NaN
4,MGSRTPESPLHAVQLRWGPRRRPPLLPLLLLLLPPPPRVGGFNLDA...,1049,0,0.0001,0,False,NaN,NaN,NaN,NaN,3D-structure . Calcium . Cell adhesion . Cell ...,114536,Homo sapiens (Human),Integrin alpha-5 (CD49 antigen-like family mem...,"cellular organisms, Eukaryota, Opisthokonta, M...",P08648,NaN,109.19,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25112,MKSYTPYFILLWSAVGIAKAAKIIIVPPIMFESHMYIFKTLASALH...,541,0,0.0025,0,False,NaN,NaN,NaN,NaN,Endoplasmic reticulum . Glycoprotein . Glycosy...,61438,Homo sapiens (Human),2-hydroxyacylsphingosine 1-beta-galactosyltran...,"cellular organisms, Eukaryota, Opisthokonta, M...",Q16880,NaN,113.56,NaN,NaN
25113,MAARTGHTALRRVVSGCRPKSATAAGAQAPVRNGRYLASCGILMSR...,238,0,0.0001,0,False,NaN,NaN,NaN,NaN,Alternative splicing . Membrane . Mitochondrio...,27071,Homo sapiens (Human),"Coenzyme Q-binding protein COQ10 homolog B, mi...","cellular organisms, Eukaryota, Opisthokonta, M...",Q9H8M1,NaN,113.74,NaN,NaN
25114,MAAVQMDPELAKRLFFEGATVVILNMPKGTEFGIDYNSWEVGPKFR...,384,0,0.0028,0,False,NaN,NaN,NaN,NaN,3D-structure . Acetylation . mRNA processing ....,43472,Homo sapiens (Human),Protein AAR2 homolog (AAR2 splicing factor hom...,"cellular organisms, Eukaryota, Opisthokonta, M...",Q9Y312,NaN,113.21,NaN,NaN
25115,MRDLLQYIACFFAFFSAGFLIVATWTDCWMVNADDSLEVSTKCRGL...,235,0,0.0054,0,False,NaN,NaN,NaN,NaN,Cell junction . Cell membrane . Disease varian...,26078,Homo sapiens (Human),Claudin-16 (Paracellin-1) (PCLN-1),"cellular organisms, Eukaryota, Opisthokonta, M...",Q9Y5I7,NaN,110.97,NaN,NaN


In [29]:
if SAVE_CV_MISTAKE_PREDS:
    df_preds.to_csv("mistake_preds_4cv_650+len_lr_2024.csv",index=False)
    df_preds.to_csv("./outputs/mistake_preds_4cv_650+len_lr_2024.csv.gz",index=False,compression="gzip")

In [30]:
display(df_preds[df_preds["mistake"]].sort_values(["model_pred_proba_vir"],ascending=False).iloc[0:10,0:4])

,Sequence,Length,virus,model_pred_proba_vir
22581,MQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ...,80,0,1.0000
17012,MDSYSAKIRANLVCRRSTDPSIRVTFSSRSLGSLPAFAMFRSSRPS...,81,0,0.9997
8265,MGCIQSITCKARIRRENIVVYDVCATIDQCPTRIEETSPIVLRYKT...,261,0,0.9997
3121,MTILFLTMVISYFGCMKAAPMKEANIRGQGGLAYPGVRTHGTLESV...,247,0,0.9995
2458,MSTSRKLKSHGMRRSKSRSPHKGVKRGGSKRKYRKGNLKSRKRGDD...,55,0,0.9994
6567,MGDPNSRKKQALNRLRAQLRKKKESLADQFDFKMYIAFVFKEKKKK...,229,0,0.9989
5364,MYGKIIFVLLLSGIVSISASSTTGVAMHTSTSSSVTKSYISSQTNG...,78,0,0.9987
17134,MAECPTLGEAVTDHPDRLWAWEKFVYLDEKQHAWLPLTIEIKDRLQ...,114,0,0.9986
2498,MANTQLDHLHYTTEFTRNDLLIICKKFNLMLMDEDIISLLAIFIKM...,102,0,0.9984
24540,MLGARVAAHLDALGPLVPYVPPPLLPSMFYVGLFFVNVLILYYAFL...,95,0,0.9983


In [31]:
df_preds.groupby("Baltimore")["mistake"].mean()

Baltimore
dsDNA       0.060394
dsDNA-RT    0.027778
dsRNA       0.005587
ssDNA       0.062016
ssRNA       0.047198
ssRNA-RT    0.108108
Name: mistake, dtype: float64